# Practise

### Dataset for practise

In [1]:
import pandas as pd
import numpy as np

np.random.seed(123)

# Generate numerical columns with NaN values
num1 = np.random.randn(500)
num2 = np.random.uniform(10, 50, size=500)
num3 = np.random.randint(1, 6, size=500).astype('float')
num1[np.random.choice(500, 60, replace=False)] = np.nan
num2[np.random.choice(500, 60, replace=False)] = np.nan
num3[np.random.choice(500, 60, replace=False)] = np.nan

# Generate messy categorical/text columns
cat1 = np.random.choice(['apple', 'banana', 'grape', np.nan], size=500, p=[0.3, 0.3, 0.3, 0.1])
cat2 = np.random.choice(['A', 'B', 'C', 'D'], size=500)
cat3 = np.random.choice(['Yes', 'No', np.nan], size=500, p=[0.45, 0.45, 0.10])

# Combine into DataFrame and add untidiness
df_untidy = pd.DataFrame({
    'Score': num1,
    'Height_cm': num2,
    'Rating': num3,
    'Fruit': cat1,
    'Group': cat2,
    'IsActive': cat3
})

# Add untidy issues:
df_untidy.loc[df_untidy.sample(frac=0.15, random_state=1).index, 'Height_cm'] = \
    df_untidy['Height_cm'].dropna().astype(str) + 'cm'   # Mix data type in Height_cm

df_untidy.loc[df_untidy.sample(frac=0.15, random_state=2).index, 'Rating'] = \
    'Rating: ' + df_untidy['Rating'].dropna().astype(str) # Prefix string for some ratings

df_untidy.head()


/tmp/ipython-input-3058238868.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['31.40565600060867cm' '23.723443878559706cm' '48.6944754257072cm'
 '47.7445897977863cm' '12.199264419087633cm' '28.255294989080216cm'
 '49.46319556401813cm' '24.62082473109778cm' '14.069842039264948cm'
 '30.047591231707173cm' '39.596908878071915cm' '47.903273405497885cm'
 '18.383678364591226cm' '20.573119172482315cm' '26.918929348570554cm'
 '49.60956000856316cm' '42.872544601068384cm' '47.28354972175643cm' nan
 nan '36.95711914375434cm' nan '25.124837252276944cm'
 '41.69467488795425cm' '28.67398840853555cm' '31.546329651552767cm'
 '14.629781437963878cm' '10.509502547356245cm' '18.213990989134608cm'
 '24.690622519316705cm' '23.881006122675878cm' '16.30134384410546cm' nan
 '29.66207244015815cm' nan '45.91010976816148cm' nan
 '31.599083317303908cm' '25.60501244185592cm' '43.06120649429532cm' nan
 '12.847775069640711cm' '12.6

,Score,Height_cm,Rating,Fruit,Group,IsActive
0,-1.085631,NaN,2.0,banana,D,Yes
1,0.997345,16.480034,Rating: 5.0,apple,A,No
2,0.282978,49.244711,NaN,banana,B,No
3,-1.506295,NaN,3.0,grape,D,nan
4,-0.578600,31.599083317303908cm,NaN,banana,C,No


- Q1. Identify columns with missing values and demonstrate at least two methods for imputing or filling these missing values (e.g., mean for numerics, mode for categoricals).

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer

# 1. Identify columns with missing values
missing_report = df_untidy.isnull().sum()
print("Missing values per column:\n", missing_report)

# 2. Imputation for numeric column: Score
imputer_mean = SimpleImputer(strategy='mean')
df_untidy['Score_mean_filled'] = imputer_mean.fit_transform(df_untidy[['Score']]).ravel()

imputer_median = SimpleImputer(strategy='median')
df_untidy['Score_median_filled'] = imputer_median.fit_transform(df_untidy[['Score']]).ravel()

# 3. Imputation for categorical column: Fruit
imputer_mode = SimpleImputer(strategy='most_frequent')
df_untidy['Fruit_mode_filled'] = imputer_mode.fit_transform(df_untidy[['Fruit']]).ravel()

# Show sample of results
df_untidy[['Score','Score_mean_filled','Score_median_filled','Fruit','Fruit_mode_filled']].head(10)

Missing values per column:
 Score        60
Height_cm    60
Rating       60
Fruit         0
Group         0
IsActive      0
dtype: int64


,Score,Score_mean_filled,Score_median_filled,Fruit,Fruit_mode_filled
0,-1.085631,-1.085631,-1.085631,banana,banana
1,0.997345,0.997345,0.997345,apple,apple
2,0.282978,0.282978,0.282978,banana,banana
3,-1.506295,-1.506295,-1.506295,grape,grape
4,-0.578600,-0.578600,-0.578600,banana,banana
5,1.651437,1.651437,1.651437,grape,grape
6,-2.426679,-2.426679,-2.426679,grape,grape
7,-0.428913,-0.428913,-0.428913,apple,apple
8,1.265936,1.265936,1.265936,apple,apple
9,-0.866740,-0.866740,-0.866740,grape,grape


- Q2.  Identify columns with non-numeric (categorical) data and convert them into a numeric format using encoding techniques such as one-hot encoding or label encoding.

In [3]:
from sklearn.preprocessing import LabelEncoder

# 1. Identify non-numeric (categorical) columns
categorical_cols = df_untidy.select_dtypes(include=['object']).columns
print("Categorical columns:", list(categorical_cols))

# 2. Label Encoding (example: Group column)
le = LabelEncoder()
df_untidy['Group_label'] = le.fit_transform(df_untidy['Group'].fillna('Unknown'))

# 3. One-Hot Encoding (example: Fruit_mode_filled column from Q1)
df_untidy = pd.get_dummies(df_untidy, columns=['Fruit_mode_filled'], prefix='Fruit')

# Show first rows with new encoded columns
df_untidy[['Group','Group_label'] + [col for col in df_untidy.columns if 'Fruit_' in col]].head(10)

Categorical columns: ['Height_cm', 'Rating', 'Fruit', 'Group', 'IsActive', 'Fruit_mode_filled']


,Group,Group_label,Fruit_apple,Fruit_banana,Fruit_grape,Fruit_nan
0,D,3,False,True,False,False
1,A,0,True,False,False,False
2,B,1,False,True,False,False
3,D,3,False,False,True,False
4,C,2,False,True,False,False
5,C,2,False,False,True,False
6,B,1,False,False,True,False
7,B,1,True,False,False,False
8,B,1,True,False,False,False
9,A,0,False,False,True,False


- Q3. Detect any columns in the DataFrame that contain mixed data types (such as numbers stored as strings or strings with prefixes). Write code to clean and convert these columns to appropriate, consistent types.

In [4]:
# 1. Check dtypes to detect suspicious (object) columns
print("Data types before cleaning:\n", df_untidy.dtypes)

# 2. Clean Height_cm → remove "cm" and convert to numeric
df_untidy['Height_cm_clean'] = (
    df_untidy['Height_cm']
    .astype(str)                          # ensure all values are strings
    .str.replace('cm','', regex=False)    # remove text 'cm'
)

# Convert to numeric (invalid parsing → NaN)
df_untidy['Height_cm_clean'] = pd.to_numeric(df_untidy['Height_cm_clean'], errors='coerce')

# 3. Clean Rating → remove "Rating: " prefix and convert to numeric
df_untidy['Rating_clean'] = (
    df_untidy['Rating']
    .astype(str)
    .str.replace('Rating: ','', regex=False)
)

df_untidy['Rating_clean'] = pd.to_numeric(df_untidy['Rating_clean'], errors='coerce')

# 4. Check again
print("Data types after cleaning:\n", df_untidy[['Height_cm_clean','Rating_clean']].dtypes)
df_untidy[['Height_cm','Height_cm_clean','Rating','Rating_clean']].head(10)

Data types before cleaning:
 Score                  float64
Height_cm               object
Rating                  object
Fruit                   object
Group                   object
IsActive                object
Score_mean_filled      float64
Score_median_filled    float64
Group_label              int64
Fruit_apple               bool
Fruit_banana              bool
Fruit_grape               bool
Fruit_nan                 bool
dtype: object
Data types after cleaning:
 Height_cm_clean    float64
Rating_clean       float64
dtype: object


,Height_cm,Height_cm_clean,Rating,Rating_clean
0,NaN,NaN,2.0,2.0
1,16.480034,16.480034,Rating: 5.0,5.0
2,49.244711,49.244711,NaN,NaN
3,NaN,NaN,3.0,3.0
4,31.599083317303908cm,31.599083,NaN,NaN
5,NaN,NaN,1.0,1.0
6,45.224286,45.224286,3.0,3.0
7,25.65266,25.652660,3.0,3.0
8,36.253728,36.253728,5.0,5.0
9,NaN,NaN,2.0,2.0


- Q4. Apply scaling and/or normalization techniques (such as Min-Max Scaling and Standardization) to the numerical columns to prepare them for downstream machine learning tasks.

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 1. Select numeric columns
numeric_cols = df_untidy.select_dtypes(include=['int64','float64']).columns
print("Numeric columns before scaling:\n", list(numeric_cols))

# Example: scale the "Score_mean_filled" column (from Q1 imputation)
scaler_minmax = MinMaxScaler()
scaler_std = StandardScaler()

df_untidy['Score_minmax'] = scaler_minmax.fit_transform(df_untidy[['Score_mean_filled']]).ravel()
df_untidy['Score_standard'] = scaler_std.fit_transform(df_untidy[['Score_mean_filled']]).ravel()

# 2. (Optional) scale other cleaned numeric columns if needed
# e.g., Height_cm_clean and Rating_clean
df_untidy['Height_minmax'] = scaler_minmax.fit_transform(df_untidy[['Height_cm_clean']].fillna(0)).ravel()
df_untidy['Height_standard'] = scaler_std.fit_transform(df_untidy[['Height_cm_clean']].fillna(0)).ravel()

df_untidy['Rating_minmax'] = scaler_minmax.fit_transform(df_untidy[['Rating_clean']].fillna(0)).ravel()
df_untidy['Rating_standard'] = scaler_std.fit_transform(df_untidy[['Rating_clean']].fillna(0)).ravel()

# 3. Show results
df_untidy[['Score_mean_filled','Score_minmax','Score_standard',
           'Height_cm_clean','Height_minmax','Height_standard',
           'Rating_clean','Rating_minmax','Rating_standard']].head(10)


Numeric columns before scaling:
 ['Score', 'Score_mean_filled', 'Score_median_filled', 'Group_label', 'Height_cm_clean', 'Rating_clean']


,Score_mean_filled,Score_minmax,Score_standard,Height_cm_clean,Height_minmax,Height_standard,Rating_clean,Rating_minmax,Rating_standard
0,-1.085631,0.346613,-1.145411,NaN,0.000000,-1.794374,2.0,0.4,-0.404642
1,0.997345,0.683137,1.088461,16.480034,0.330007,-0.659884,5.0,1.0,1.434638
2,0.282978,0.567725,0.322344,49.244711,0.986110,1.595644,NaN,0.0,-1.630828
3,-1.506295,0.278651,-1.596549,NaN,0.000000,-1.794374,3.0,0.6,0.208452
4,-0.578600,0.428529,-0.601650,31.599083,0.632762,0.380915,NaN,0.0,-1.630828
5,1.651437,0.788812,1.789936,NaN,0.000000,-1.794374,1.0,0.2,-1.017735
6,-2.426679,0.129954,-2.583608,45.224286,0.905602,1.318877,3.0,0.6,0.208452
7,-0.428913,0.452712,-0.441118,25.652660,0.513686,-0.028438,3.0,0.6,0.208452
8,1.265936,0.726530,1.376509,36.253728,0.725969,0.701342,5.0,1.0,1.434638
9,-0.866740,0.381977,-0.910663,NaN,0.000000,-1.794374,2.0,0.4,-0.404642


- Q5. Write a function to check for and report any remaining inconsistencies (missing values, mixed types, out-of-range values) in the cleaned DataFrame. Validate that the preprocessing steps have successfully prepared the data for analysis.

In [6]:
def validate_cleaning(df):
    results = {}

    # 1. Missing values check
    results['missing_values'] = df.isnull().sum().to_dict()

    # 2. Data types check
    results['data_types'] = df.dtypes.astype(str).to_dict()

    # 3. Out-of-range check (example: Rating must be between 1 and 5)
    if 'Rating_clean' in df.columns:
        outliers = df[(df['Rating_clean'] < 1) | (df['Rating_clean'] > 5)]
        results['rating_outliers'] = len(outliers)
    else:
        results['rating_outliers'] = None

    # 4. Optional: check if categorical columns still contain numeric-looking strings
    mixed_type_columns = []
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if column has both numbers and non-numeric values
            try:
                pd.to_numeric(df[col])
            except Exception:
                mixed_type_columns.append(col)
    results['mixed_type_columns'] = mixed_type_columns

    return results

# Run validation on cleaned DataFrame
validation_report = validate_cleaning(df_untidy)
validation_report

{'missing_values': {'Score': 60,
  'Height_cm': 60,
  'Rating': 60,
  'Fruit': 0,
  'Group': 0,
  'IsActive': 0,
  'Score_mean_filled': 0,
  'Score_median_filled': 0,
  'Group_label': 0,
  'Fruit_apple': 0,
  'Fruit_banana': 0,
  'Fruit_grape': 0,
  'Fruit_nan': 0,
  'Height_cm_clean': 60,
  'Rating_clean': 60,
  'Score_minmax': 0,
  'Score_standard': 0,
  'Height_minmax': 0,
  'Height_standard': 0,
  'Rating_minmax': 0,
  'Rating_standard': 0},
 'data_types': {'Score': 'float64',
  'Height_cm': 'object',
  'Rating': 'object',
  'Fruit': 'object',
  'Group': 'object',
  'IsActive': 'object',
  'Score_mean_filled': 'float64',
  'Score_median_filled': 'float64',
  'Group_label': 'int64',
  'Fruit_apple': 'bool',
  'Fruit_banana': 'bool',
  'Fruit_grape': 'bool',
  'Fruit_nan': 'bool',
  'Height_cm_clean': 'float64',
  'Rating_clean': 'float64',
  'Score_minmax': 'float64',
  'Score_standard': 'float64',
  'Height_minmax': 'float64',
  'Height_standard': 'float64',
  'Rating_minmax': 'flo